### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [122]:
reload(bs)
reload(dl)
reload(fit)

<module 'fitter.fitter' from '../fitter\fitter.pyc'>

### Load data

In [123]:
data_loader = dl.data_loader()

fit_data = data_loader.get_fit_data()
prior = data_loader.get_prior()

### Bootstrap

In [124]:
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=1, bs_N=50)
#bootstrapper.plot_parameters(xy_parameters=['a', 'm_pion'], color_parameter='V')
#data_loader.save_plots(bootstrapper)
#bootstrapper.plot_fit()

In [127]:
import time
t0 = time.time()
print bootstrapper
t1 = time.time()
print "Total time (s): ", t1 - t0

#squared = lambda x : x**2
#bootstrapper.plot_fit(xfcn=squared, xlabel='$m_{\pi}^2$ (MeV)$^2$')
#bootstrapper.plot_fit()

print bootstrapper.fits[0]



Fitting to order 1 
Fitted/[Experimental] values at physical point:

		F_K / F_pi = 1.06(16) [1.201(46)]

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
     l_ju 0              -1.1 (3.4)       l_ss 0               0.52 (24)
          1               1.0 (1.0)            1               1.0 (1.0)
     l_sj 0               1.0 (3.1)       l_rs 0              -2.7 (3.7)
          1               1.0 (1.0)            1               1.0 (1.0)
      l_x 0              -0.40 (18)       l_ru 0               0.8 (1.8)
          1               1.0 (1.0)            1               1.0 (1.0)
     l_pi 0              -0.66 (83)     l_5lam 0             -0.002 (13)
          1               1.0 (1.0)            1             0.0069 (34)
Total time (s):  0.00600004196167
Least Square Fit:
  chi2/dof [dof] = 17 [16]    Q = 1.7e-47    logGBF = -72.113

Parameters:
         l_5lam   -0.0056 (16)     